In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('filtered-pre-mirnas.csv')
df

,MFE,sequence,MFEIS,class
0,-45.7,CAUUAUUUUGGCAUCUUUUUCUGCUUCCACAAGUGCUUUUGUGGAA...,-0.932653,Real pre-miRNA
1,-50.7,UGAACUUGAUGAAAGGAUUGAUAAAGUACUUAAUGAGAAGUGCUCU...,-0.938889,Real pre-miRNA
2,-44.6,UUCGCACAAAUGUCUACUUAGCUGUCUUUGAUAAAAAUCUCUAUGA...,-0.910204,Real pre-miRNA
3,-50.4,UGCUUAUUACAGAGCAAGGGCUGGUGAAGCUGCUAACUUUUGUGCA...,-0.916364,Real pre-miRNA
4,-42.9,AACCUGCUUCAAGAGAGCUUAAAGUUACAUUUUUCCCUGACUUAAA...,-0.893750,Real pre-miRNA
5,-44.9,GGUUCCAUCCAUCUUUACUUUGAUAAAGCUGGUCAAAAGACUUAUG...,-0.935417,Real pre-miRNA
6,-41.3,CUUUUGAAGAAGCUGCGCUGUGCACCUUUUUGUUAAAUAAAGAAAU...,-0.860417,Real pre-miRNA
7,-39.2,UUACUGCAAAAUGGUAUGAAUGGACGUACCAUAUUGGGUAGUGCUU...,-0.911628,Real pre-miRNA
8,-45.4,UAGUUUUAAGGAAUUACUUGUGUAUGCUGCUGACCCUGCUAUGCAC...,-0.873077,Real pre-miRNA
9,-39.9,AGCGAUUAUGACUACUAUCGUUAUAAUCUACCAACAAUGUGUGAUA...,-0.997500,Real pre-miRNA


In [9]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=7139ff5c07327e5dd4deb2418e2dcb8cd3b31419fb225d16e64779f76f85845f
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [43]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [89]:
import re

def parse_bayes_output(soup_html):
    stem_re = "\d' *stem"
    rna_re = "[AUGC]+"

    seqs = [el.text for el in html_soup.select('tr')[2:]]
    seqs = [(re.findall(rna_re, mirna), re.findall(stem_re, mirna)) for mirna in seqs]
    seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
#     output = []
#     for strand_match, sequence_match in seqs:
#         strand = strand_match[0]
#         sequence = sequence_match[0]
#         print(sequence)
#         if len(strand) > 0 and len(sequence) > 0:
#             output.append((strand, sequence))
#         seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
    return list(set(seqs))

In [90]:
mature_bayes_url = "http://mirna.imbb.forth.gr/cgi-bin/MatureBayes/maturebayes.py"
mature_mirnas = []

for i in tqdm(range(df.shape[0])):
    premirna_seq = df.iloc[i]['sequence']
    payload = {"single_seq": premirna_seq}
    
    response = requests.post(mature_bayes_url, payload)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    new_mature_mirnas = parse_bayes_output(html_soup)
    mature_mirnas += new_mature_mirnas
    
mature_mirnas = list(set(mature_mirnas))
print(f'Found {len(mature_mirnas)} mature miRNA candidates.')

  0%|          | 0/27 [00:00<?, ?it/s]

Found 75 mature miRNA candidates.


In [97]:
mature_mirnas = pd.DataFrame(mature_mirnas, columns=['sequence', 'strand'])
mature_mirnas

,sequence,strand
0,AUGUCUACUUAGCUGUCUUUGA,5'
1,CUUCUGGUAAUCUAUUACUAGA,5'
2,UACCAUAUUGGGUAGUGCUUUA,3'
3,UACUAAAUUAAAUGAUCUCUGC,5'
4,AGUGUUUUAAAUGAUAUCCUUU,3'
...,...,...
70,GAGGUGAUGAAGUCAGACAAAU,5'
71,UGACUUAAAUGGUGAUGUGGUG,5'
72,UCAUUACUUCAGGUGAUGGCAC,5'
73,UGUUGAUGCACUAUGUGAGAAG,3'


In [98]:
mature_mirnas.to_csv('mature-mirnas-from-mirnafold.csv', index=False)